In [1]:
from gpu_rocket import torchrocket
import numpy as np
import torch

c:\Program Files\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Program Files\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Program Files\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
num_examples = 1
num_channels = 20
timesteps = 100

cpu


In [3]:
# either: 
# -use the kernels from sktime (and then convert to 32bit if you want 32bit gpu kernels)
# -generate them yourself
# -or use the one I have edited from sktime to make 32bit 

kernels = torchrocket._generate_kernels_sktime_32bit_version(timesteps, 10_000, num_channels, seed=None)
# Note: it is required to convert each of these np arrays to lists if you want to script the model (script doesnt work with np)
# e.g. channel_indices = channel_indices.tolist() etc

In [4]:
X = np.random.rand(num_examples,num_channels,timesteps).astype(np.float32)
X_torch =torch.from_numpy(X.astype(np.float32)).to(device)

In [5]:
torch_rocket_softPPV_10k = torchrocket.TorchRocket(kernels,ppv_ver='softPPV',softppv_param=10_000.)
torch_rocket_softPPV_10k.to(device)
torch_rocket_softPPV_10k.eval()
torch_rocket_softPPV_5 = torchrocket.TorchRocket(kernels,ppv_ver='softPPV',softppv_param=5.)
torch_rocket_softPPV_5.to(device)
torch_rocket_softPPV_5.eval()
torch_rocket_heaviside_PPV = torchrocket.TorchRocket(kernels,ppv_ver='heaviside_PPV')
torch_rocket_heaviside_PPV.to(device)
torch_rocket_heaviside_PPV.eval()
print(f'')

In [6]:
with torch.no_grad():
    out_softPPV_10k = torch_rocket_softPPV_10k(X_torch)
    out_softPPV_5 = torch_rocket_softPPV_5(X_torch)
    out_heaviside_PPV = torch_rocket_heaviside_PPV(X_torch)

In [7]:
# compare to sktime implementation
from sktime.transformations.panel.rocket import _rocket
out_sktime_rocket = _rocket._apply_kernels(X, kernels)

In [8]:
print(f'out_softPPV shape: {out_softPPV_10k.shape}')
print(f'out_softPPV shape: {out_softPPV_5.shape}')
print(f'out_heaviside_PPV shape: {out_heaviside_PPV.shape}')
print(f'out_sktime_rocket shape: {out_sktime_rocket.shape}')

out_softPPV shape: torch.Size([20000])
out_softPPV shape: torch.Size([20000])
out_heaviside_PPV shape: torch.Size([20000])
out_sktime_rocket shape: (1, 20000)


In [9]:
for i in range(len(out_softPPV_5)):
    print(f'{i}: {out_softPPV_5[i]},{out_softPPV_10k[i]},{out_heaviside_PPV[i]},{out_sktime_rocket[0][i]}')

0: 0.3727497160434723,0.568965494632721,0.568965494632721,0.568965494632721
1: 3.0221095085144043,3.0221095085144043,3.0221095085144043,3.0221097469329834
2: 0.3705603778362274,0.4800001382827759,0.47999998927116394,0.47999998927116394
3: 3.951538324356079,3.951538324356079,3.951538324356079,3.951538562774658
4: 0.5446598529815674,0.7599999904632568,0.7599999904632568,0.7599999904632568
5: 3.7534024715423584,3.7534024715423584,3.7534024715423584,3.7534024715423584
6: 0.4471263289451599,0.699999988079071,0.699999988079071,0.699999988079071
7: 3.115635871887207,3.115635871887207,3.115635871887207,3.1156363487243652
8: 0.6180526614189148,0.7333333492279053,0.7333333492279053,0.7333333492279053
9: 6.165524482727051,6.165524482727051,6.165524482727051,6.165523529052734
10: 0.300923615694046,0.3804347813129425,0.3804347813129425,0.3804347813129425
11: 4.89743709564209,4.89743709564209,4.89743709564209,4.897437572479248
12: 0.5742495656013489,0.6714285612106323,0.6714285612106323,0.6714285612